# Query the Database

Şimdi bir `ecommerce` database üzerinde gelişmiş SQL sorguları yazma zamanı!

## Data
Bu URL'de bulunan `ecommerce.sqlite` database ile çalışacağız:  
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite`

Dosyayı indirmek için aşağıdaki hücreyi çalıştırın.


In [1]:
!curl https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite > data/ecommerce.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9216  100  9216    0     0  13539      0 --:--:-- --:--:-- --:--:-- 13552


## Setup

Pandas ve sqlite3 ihtiyacımız olan her şey :-)


In [14]:
import pandas as pd
from sqlite3 import connect

In [15]:
import sqlite3

conn = sqlite3.connect('data/ecommerce.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
conn.close()

tables



[('Customers',), ('Shippers',), ('Employees',), ('Orders',), ('OrderDetails',)]

## Orders

👉 Tüm orders kayıtlarını, tüm column'ları göstererek alın.

In [16]:
#  Her sütunu gösteren siparişlerin listesini  return et
query_orders = "select * from  orders "

In [17]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_orders,
        con=conn
    )
df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,FreightCharge
0,1,1,1,2012-01-04,2012-01-09,2012-01-05,1,3.75
1,2,2,2,2012-01-27,2012-02-01,2012-01-28,1,7.25
2,3,4,1,2012-02-19,2012-02-24,2012-02-23,2,5.50
3,4,2,4,2012-03-13,2012-03-18,2012-03-14,2,13.50
4,5,4,2,2012-04-05,2012-04-10,2012-04-06,3,8.75


Sonuç beklediğiniz gibi göründüğünde, sorgunuzu test etmek için aşağıdaki hücreyi çalıştırın.

In [18]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'query_orders',
    query=query_orders
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/senemyilmaz/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/senemyilmaz/data-query-the-db/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 2 items

test_query_orders.py::TestQueryOrders::test_first_element PASSED         [ 50%]
test_query_orders.py::TestQueryOrders::test_length_list PASSED           [100%]

============================== 2 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/query_orders.pickle

git commit -m 'Completed query_orders step'

git push origin master



## Orders range

👉 Verilen iki tarih arasında yapılan tüm orders kayıtlarını, OrderDate değerine göre artan şekilde alın (date_from hariç, date_to dahil).


In [27]:
# date_from ve date_to tarihleri arasında (date_from hariç ve date_to dahil)
# OrderDate sütununu gösteren siparişlerin listesini return et
query_orders_range = """
    SELECT *
    FROM orders
    WHERE OrderDate > ?
    AND OrderDate <= ?
"""

👉 Bu sefer, sorguyu denemek için veriyi bir dataframe'e yükleyecek kodu kendiniz yazın. Dinamik olmasına dikkat edin: başlangıç ve bitiş tarihlerini kolayca değiştirebilmek istiyoruz!

Nasıl yapacağınızdan emin değil misiniz? Önceki challenge’tan ilham alın!

In [28]:
start_date = '2013-01-01'
end_date = '2013-01-31'

with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_orders_range,
        con=conn,
        params=[start_date, end_date]
    )
df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,FreightCharge
0,17,5,1,2013-01-06,2013-01-11,2013-01-07,3,6.25
1,18,3,3,2013-01-29,2013-02-03,2013-01-30,1,10.75


Sonuç beklediğiniz gibi göründüğünde, sorgunuzu test etmek için aşağıdaki hücreyi çalıştırın.

In [29]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'get_orders_range',
    query=query_orders_range
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/senemyilmaz/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/senemyilmaz/data-query-the-db/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 4 items

test_get_orders_range.py::TestGetOrdersRange::test_len_results PASSED    [ 25%]
test_get_orders_range.py::TestGetOrdersRange::test_results_0 PASSED      [ 50%]
test_get_orders_range.py::TestGetOrdersRange::test_results_1 PASSED      [ 75%]
test_get_orders_range.py::TestGetOrdersRange::test_type_results PASSED   [100%]

============================== 4 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/get_orders_range.pickle

git commit -m 'Completed get_orders_range step'

git push origin master



## Waiting time

👉 Delivery time değerine göre (en küçük timedelta'dan en büyüğüne doğru) artan sırada tüm orders kayıtlarını alın.

İpucu: "sqlite julianday" şeklinde arama yapın.

In [39]:
#  Her sütunu gösteren tüm siparişlerin listesini alın
# ve OrderDate ile ShippedDate arasındaki gün sayısını gösteren ek bir TimeDelta sütunu hesaplayın
# TimeDelta'ya göre artan sırada sıralayın
query_waiting_time = """
    SELECT *, julianday(ShippedDate) - julianday(OrderDate) AS TimeDelta from orders order by TimeDelta

"""

Sonuç beklediğiniz gibi göründüğünde, sorgunuzu denemek ve test etmek için aşağıdaki hücreleri çalıştırın.

In [40]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_waiting_time,
        con=conn
    )
df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,FreightCharge,TimeDelta
0,1,1,1,2012-01-04,2012-01-09,2012-01-05,1,3.75,1.0
1,2,2,2,2012-01-27,2012-02-01,2012-01-28,1,7.25,1.0
2,4,2,4,2012-03-13,2012-03-18,2012-03-14,2,13.50,1.0
3,5,4,2,2012-04-05,2012-04-10,2012-04-06,3,8.75,1.0
4,6,3,3,2012-04-28,2012-05-03,2012-04-29,2,11.00,1.0


In [41]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'get_waiting_time',
    query=query_waiting_time
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/senemyilmaz/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/senemyilmaz/data-query-the-db/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 4 items

test_get_waiting_time.py::TestGetWaitingTime::test_first_result PASSED   [ 25%]
test_get_waiting_time.py::TestGetWaitingTime::test_last_result PASSED    [ 50%]
test_get_waiting_time.py::TestGetWaitingTime::test_size_list PASSED      [ 75%]
test_get_waiting_time.py::TestGetWaitingTime::test_type_results PASSED   [100%]

============================== 4 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/get_waiting_time.pickle

git commit -m 'Completed get_waiting_time step'

git push origin master



## Key learning points

- Bir database üzerinde query yazmak için Pandas kullanın
- Dinamik queries yazmak için parameter substitution kullanın
- Tarihlerle çalışın ve bu tarihlerle hesaplamalar yapın